In [1]:
# 1) Mean/Median Age (Age_Days) - Descriptive Statistics Mean/IQR
# 2) Mean/Median Weight (WT_KGS) - Descriptive Statistics Mean/IQR
# 3) Gender distribution (GENDER) - significant differences? - One Way Chi Squre
# 4) Most common Breeds (BREED) - One Way Chi Squre
# 5) Most common shunts (SHUNT) - One Way Chi Squre
# 6) Most common portal vein origin (PORTAL_V) - One Way Chi Squre
# 7) Most common systemic vein insertion (SYSTEM_V) - One Way Chi Squre
# 8) Most common contributing vessels (CONTRB_V) - One Way Chi Squre
# 9) PP score distribution (PP_Score) - One Way Chi Squre and Descriptive Statistics Mean/IQR (age, wt, and shunt_ty are set as continuous exploratory)

# Cross tab frequencies and ANOVA
# HOSPITAL COMPARISONS (HOSPITAL):
# 10) Do the above values differ AMONG HOSPITALS (HOSPITAL) - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# COUNTRY COMPARISONS (COUNTRY):
# 11) Do the above values differ AMONG COUNTRIES (COUNTRY)  - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# BREED COMPARISONS (BREED): [5 MOST COMMON BREEDS]
# 12) What is Gender distribution (GENDER) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 13) What is Age distribution (Age_Days) comparison among 5 most common breeds (BREED)? - Sub ANOVA limited to breeds 61, 38, 53, 35, 47
# 14) What are 5 most common Shunts  (SHUNT) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 15) What is PP Score (PP_Score) comparison among 5 most common breeds (BREED)? # Probably best looked at categorically with two-way
# comprehensive chi-square for contingency table first time (Comprehensive pairwise comparisons reveals)


# Regression and ANOVA
# AGE COMPARISONS (Age_Days):
# 16) Does AGE (Age_Days) correlate with Breed (BREED) - RQ 13 handles this with age as DV and Breed (Top 5) as IV with 5 class levels fo breed
# 17) Does AGE (Age_Days) correlate with PP Score (PP_Score) - Standard ANOVA but similar issue with shunt type having a lot of cats, see 18
# 18) Does AGE (Age_Days) correlate with Shunt (SHUNT) - There are 44 different shunt types (shunt type is not ordinal) and age is continuous. 
#  An ANOVA would 
# not yield meaningful results iwth the 44 goups giving a mean age for pairise comparisons.  We could contribute collapsing shunt and 
# bucketing age.


# SHUNT COMPARISONS (SHUNT) ANOVA:
# 19) Does Shunt (SHUNT) correlate with Weight (WT_KGS)    [Above and below mean/median?] - see 20
# 20) Does Shunt (SHUNT) correlate with PP Score (PP_Score) There are 44 different shunt types (non-ordinal) and 11 (with one missing) pp Score.  
# This could be a 
# either a huge cross tab or a difficlt it interpret ANOVA.   Need to define DV/IV an consider collpasing (top 5?) Shunt Type. See 17 and 18

# Breed ANOVA and Cross Tab
# These (21, 22, 23) are all caegorical with multiple and in most cases non-ordinal categorization making inelgible for regression and 
# multiple categories difficult to interpret with thin data classes for ANOVA.  Suggest first round (Comprehensive pairwise comparisons reveals)
# 21) Does PV origin (PORTAL_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 22) Does systemic v insertion (SYSTEM_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 23) Do contributing vessels (CONTRB_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square

In [2]:
from tabulate import tabulate
import os, saspy, itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime, timedelta
today_dt = datetime.today().strftime("%Y-%m-%d")
now = datetime.now()

print("Today's production date and time : ", now.strftime("%Y-%m-%d %H:%M:%S"))

#Stats Import
from scipy.stats import wilcoxon 
from scipy.stats import shapiro 
from scipy.stats import kstest
# from numpy.random import seed
# from numpy.random import math

import pingouin as pg
from pingouin import pairwise_ttests, pairwise_tukey, pairwise_gameshowell
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import scipy.stats as stats
# from scipy import stats # Might need as is for ttest proportiality z or tdist
import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.formula.api import ols
from scipy.stats import wilcoxon
from statsmodels.stats.proportion import proportions_ztest



import io
from io import BytesIO
import base64


from operator import concat

# sys.path.append(os.path.join(sys.path[0],'C:\Junk\lamb_consulting'))
sys.path.append(os.path.join(sys.path[0],'U:\lamb_consulting'))


# import cox, means, regressions

from lamb_consulting import *

from lamb_consulting.means import means_desc, means_desc_class, aov
from lamb_consulting.freqs import freqs, chi_contingency, chi_oneway
from lamb_consulting.export import export_to_Excel
# from lamb_consulting.regressions import ols, regression_plot
from lamb_consulting.cox import cox
from lamb_consulting.kaplan_meier import km,km_plots


def freqme(ds,var1):
    f=ds[var1].value_counts(dropna=False)
    p=ds[var1].value_counts(normalize=True)
    df=pd.concat({f,p}, axis=1, keys=['frequency', 'percent'])
    df["cumfrequency"] = df["frequency"].cumsum()
    df["cumpercent"] = df["percent"].cumsum()
    return df



# For figure image to base 64 writing

def fig_to_base64(fig):
    img = io.BytesIO()
    fig.savefig(img, format='png',
                bbox_inches='tight')
    img.seek(0)

    return base64.b64encode(img.getvalue())

Today's production date and time :  2023-01-16 14:57:38


In [3]:
print(os.getcwd())
# tpath = "C:/Junk/Weisse/EHPSS/"
tpath = "U:/Fox/Weisse/EHPSS/"
fn="EHPSS20221207"

extension='.xlsx'
file = tpath+fn+extension

df_in = pd.read_excel((file), sheet_name='Data')

# Filter out age lt 90d 
# df = df_in.loc[(df_in['age_d']>=90)]
# Treating lt 90 as a class
df = df_in.copy()
df['age_d_ref'] = df['age_d']
# Set ages lt 90d and gt 10 yr to nan
df['age_d'] = np.where((df['age_d'] <= 90) | (df['age_d'] >= 3650), np.nan, df['age_d'])

# print(df_in.loc[515:525,['age_d','dob']])
# print(df.loc[515:525,['age_d','dob']])
# print(df['age_d'].min())
# print(df['age_d'].max())


df['contrb_v'].fillna(method='ffill')
df['wt_kg'].fillna(method='ffill')

# df_90_730 = df.loc[(df['age_d'] >= 90) & (df['age_d'] <= 730) ]
# df_730_1095 = df.loc[(df['age_d'] > 730) & (df['age_d'] <= 1095)]
# df_1095 = df.loc[(df['age_d' ] > 1095)]

df_90_912p5 = df.loc[(df['age_d'] >= 90) & (df['age_d'] <= 912.5) ]
df_912p5_1825 = df.loc[(df['age_d'] > 912.5) & (df['age_d'] <= 1825)]
df_1825_3560 = df.loc[(df['age_d'] > 1825) & (df['age_d'] <= 3560)]
df_90_3560 = df.loc[(df['age_d'] >= 90) & (df['age_d'] <= 3560) ]

# 50%        733.000000
def age_lt_90(row):
    if row["age_d"] <= 90:
        return 1
    elif row["age_d"] > 90:
        return 0
    else:
        return -1

def age_lt_730(row):
    if row["age_d"] <= 730:
        return 1
    elif row["age_d"] > 730:
        return 0
    else:
        return -1

# Note changed 1 to second condition for greater than given variable name change
def age_gt_1095(row):
    if row["age_d"] <= 1095:
        return 0
    elif row["age_d"] > 1095:
        return 1
    else:
        return -1


def age_90_730(row):
    if row["age_d"] <= 90:
        return 1
    elif row["age_d"] <= 730:
        return 2
    elif row["age_d"] > 730:
        return 3
    else:
        return -1

def age_730_1095(row):
    if row["age_d"] <= 730:
        return 1
    elif row["age_d"] <= 1095:
        return 2
    elif row["age_d"] > 1095:
        return 3
    else:
        return -1

def age_2p5_5_10yr(row):
    if row["age_d"] <= 90:
        return 0
    if row["age_d"] <= 912.5:
        return 1
    elif row["age_d"] < 1825:
        return 2
    elif row["age_d"] < 3560:
        return 3
    elif row["age_d"] > 3560:
        return 4
    else:
        return -1



# 90-912.5d  (under 2.5yrs)
# 912.5d-1825d  (2.5y-5yrs)
# 1825d-3650d  (5yrs to 10 yrs)


def male_n(row):
    if row["gender"] == 1 or row["gender"] == 2:
        return 1
    elif row["gender"] == 3 or row["gender"] == 4:
        return 0
    else:
        return -1


def male_c(row):
    if row["gender_1"] == 'MC' or row["gender_1"] == 'MI' or row["gender_1"] == 'NM' or row["gender_1"] == 'MN' or row["gender_1"] == 'ME' or row["gender_1"] == 'M':
        return "M"
    elif row["gender_1"] == 'FS' or row["gender_1"] == 'FI' or row["gender_1"] == 'FE': 
        return "F"
    else:
        return "error"



# breed_t5_list = [62, 61, 53, 47, 38, 35]
# shunty_t10_list = [11, 9, 8, 7, 6, 5, 4, 3, 2, 1]

# Will revisit this to loop datafrmes into apply statements, giving er as being passed in as a string
# my_dfs = ['df_90_730', 'df_730_1095', 'df_1095']
my_dfs = ['df_90_912p5', 'df_912p5_1825', 'df_1825_3560', 'df_90_3560']

  

df['male_n'] = df.apply(male_n,axis=1)
df['male_c'] = df.apply(male_c,axis=1)
df['age_lt_90'] = df.apply(age_lt_90,axis=1)
df['age_lt_730'] = df.apply(age_lt_730,axis=1)
df['age_gt_1095'] = df.apply(age_gt_1095,axis=1)
df['age_90_730'] = df.apply(age_90_730,axis=1)
df['age_730_1095'] = df.apply(age_730_1095,axis=1)
df['age_2p5_5_10yr'] = df.apply(age_2p5_5_10yr,axis=1) 


df_90_912p5['male_n'] = df_90_912p5.apply(male_n,axis=1)
df_90_912p5['male_c'] = df_90_912p5.apply(male_c,axis=1)
# df_90_912p5['age_lt_90'] = df_90_730.apply(age_lt_90,axis=1)
# df_90_912p5['age_lt_730'] = df_90_730.apply(age_lt_730,axis=1)
# df_90_912p5['age_gt_1095'] = df_90_730.apply(age_gt_1095,axis=1)
# df_90_912p5['age_90_730'] = df_90_730.apply(age_90_730,axis=1)
# df_90_912p5['age_730_1095'] = df_90_730.apply(age_730_1095,axis=1)
df_90_912p5['age_2p5_5_10yr'] = df_90_912p5.apply(age_2p5_5_10yr,axis=1)

df_912p5_1825['male_n'] = df_912p5_1825.apply(male_n,axis=1)
df_912p5_1825['male_c'] = df_912p5_1825.apply(male_c,axis=1)
# df_912p5_1825['age_lt_90'] = df_912p5_1825.apply(age_lt_90,axis=1)
# df_912p5_1825['age_lt_730'] = df_912p5_1825.apply(age_lt_730,axis=1)
# df_912p5_1825['age_gt_1095'] = df_912p5_1825.apply(age_gt_1095,axis=1)
# df_912p5_1825['age_90_730'] = df_912p5_1825.apply(age_90_730,axis=1)
# df_912p5_1825['age_730_1095'] = df_912p5_1825.apply(age_730_1095,axis=1)
df_912p5_1825['age_2p5_5_10yr'] = df_912p5_1825.apply(age_2p5_5_10yr,axis=1)

df_1825_3560['male_n'] = df_1825_3560.apply(male_n,axis=1)
df_1825_3560['male_c'] = df_1825_3560.apply(male_c,axis=1)
# df_1095['age_lt_90'] = df_1095.apply(age_lt_90,axis=1)
# df_1095['age_lt_730'] = df_1095.apply(age_lt_730,axis=1)
# df_1095['age_gt_1095'] = df_1095.apply(age_gt_1095,axis=1)
# df_1095['age_90_730'] = df_1095.apply(age_90_730,axis=1)
# df_1095['age_730_1095'] = df_1095.apply(age_730_1095,axis=1)
df_1825_3560['age_2p5_5_10yr'] = df_1825_3560.apply(age_2p5_5_10yr,axis=1)

df_90_3560['male_n'] = df_90_3560.apply(male_n,axis=1)
df_90_3560['male_c'] = df_90_3560.apply(male_c,axis=1)
df_90_3560['age_2p5_5_10yr'] = df_90_3560.apply(age_2p5_5_10yr,axis=1)


cont = ['age_d', 'wt_kg', 'shunt_ty']
cont_noage = ['wt_kg', 'shunt_ty']
cat = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']

# cat_nopps = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country','age_lt_730','age_gt_1095','age_90_730','age_730_1095','age_2p5_5_10yr']
cat_nopps = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country','age_2p5_5_10yr']
# removing 'age_lt_90' since this has been removed from DS

# cont_describe = "age_d wt_kg pps_n"
# cat_describe = "male_c breed_nc shunt_ty portal_v system_v contrb_v hospital country shunt_ty"


# print(df["gender_1"].value_counts())
# print(df["gender"].value_counts())

# print(df["male_c"].value_counts())
# print(df["male_n"].value_counts())

print(df.shape)
print(df_90_912p5.shape)
print(df_912p5_1825.shape)
print(df_1825_3560.shape)
print(df_90_3560.shape)



   

u:\Fox\Weisse\EHPSS


C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\loc

(1082, 35)
(592, 30)
(215, 30)
(201, 30)
(1008, 30)


C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Create Top 6 Breed dataset based of of the one-way Chi-Squre Output

In [4]:
# breed_t5_list = [62, 61, 53, 47, 38, 35]
breed_t6_list = df['breed_n'].value_counts().keys()[0:6].sort_values(ascending=False).tolist()
breed_mask = (df['breed_n'].isin(breed_t6_list)) 
breed_t6 = df.loc[breed_mask]
print(breed_t6_list)
print(df.shape)

[62.0, 61.0, 53.0, 47.0, 38.0, 35.0]
(1082, 35)


# Breed categoricals

In [5]:
# Top 6 categorical

my_dfs = ['df_90_912p5', 'df_912p5_1825', 'df_1825_3560', 'df_90_3560']


df['breed_t6'] = np.where(df['breed_n'].isin(breed_t6_list),1,0) 
df_90_912p5['breed_t6'] = np.where(df_90_912p5['breed_n'].isin(breed_t6_list),1,0) 
df_912p5_1825['breed_t6'] = np.where(df_912p5_1825['breed_n'].isin(breed_t6_list),1,0) 
df_1825_3560['breed_t6'] = np.where(df_1825_3560['breed_n'].isin(breed_t6_list),1,0) 
df_90_3560['breed_t6'] = np.where(df_90_3560['breed_n'].isin(breed_t6_list),1,0) 
# print(df[['breed_t5', 'breed_n']])
print(df['breed_t6'].value_counts())

breed_t5_list = df['breed_n'].value_counts().keys()[0:5].sort_values(ascending=False).tolist()
df['breed_t5'] = np.where(df['breed_n'].isin(breed_t5_list),1,0) 
df_90_912p5['breed_t5'] = np.where(df_90_912p5['breed_n'].isin(breed_t5_list),1,0) 
df_912p5_1825['breed_t5'] = np.where(df_912p5_1825['breed_n'].isin(breed_t5_list),1,0) 
df_1825_3560['breed_t5'] = np.where(df_1825_3560['breed_n'].isin(breed_t5_list),1,0) 
df_90_3560['breed_t5'] = np.where(df_90_3560['breed_n'].isin(breed_t5_list),1,0) 
print(df['breed_t5'].value_counts())

breed_t4_list = df['breed_n'].value_counts().keys()[0:4].sort_values(ascending=False).tolist()
df['breed_t4'] = np.where(df['breed_n'].isin(breed_t4_list),1,0) 
df_90_912p5['breed_t4'] = np.where(df_90_912p5['breed_n'].isin(breed_t4_list),1,0) 
df_912p5_1825['breed_t4'] = np.where(df_912p5_1825['breed_n'].isin(breed_t4_list),1,0) 
df_1825_3560['breed_t4'] = np.where(df_1825_3560['breed_n'].isin(breed_t4_list),1,0) 
df_90_3560['breed_t4'] = np.where(df_90_3560['breed_n'].isin(breed_t4_list),1,0) 
print(df['breed_t4'].value_counts())

breed_t3_list = df['breed_n'].value_counts().keys()[0:3].sort_values(ascending=False).tolist()
df['breed_t3'] = np.where(df['breed_n'].isin(breed_t3_list),1,0) 
df_90_912p5['breed_t3'] = np.where(df_90_912p5['breed_n'].isin(breed_t3_list),1,0) 
df_912p5_1825['breed_t3'] = np.where(df_912p5_1825['breed_n'].isin(breed_t3_list),1,0) 
df_1825_3560['breed_t3'] = np.where(df_1825_3560['breed_n'].isin(breed_t3_list),1,0) 
df_90_3560['breed_t3'] = np.where(df_90_3560['breed_n'].isin(breed_t3_list),1,0) 
print(df['breed_t3'].value_counts())

breed_t2_list = df['breed_n'].value_counts().keys()[0:2].sort_values(ascending=False).tolist()
df['breed_t2'] = np.where(df['breed_n'].isin(breed_t2_list),1,0) 
df_90_912p5['breed_t2'] = np.where(df_90_912p5['breed_n'].isin(breed_t2_list),1,0)
df_912p5_1825['breed_t2'] = np.where(df_912p5_1825['breed_n'].isin(breed_t2_list),1,0)
df_1825_3560['breed_t2'] = np.where(df_1825_3560['breed_n'].isin(breed_t2_list),1,0) 
df_90_3560['breed_t2'] = np.where(df_90_3560['breed_n'].isin(breed_t2_list),1,0) 
print(df['breed_t2'].value_counts())

breed_t1_list = df['breed_n'].value_counts().keys()[0:1].sort_values(ascending=False).tolist()
df['breed_t1'] = np.where(df['breed_n'].isin(breed_t1_list),1,0) 
df_90_912p5['breed_t1'] = np.where(df_90_912p5['breed_n'].isin(breed_t1_list),1,0) 
df_912p5_1825['breed_t1'] = np.where(df_912p5_1825['breed_n'].isin(breed_t1_list),1,0) 
df_1825_3560['breed_t1'] = np.where(df_1825_3560['breed_n'].isin(breed_t1_list),1,0)
df_90_3560['breed_t1'] = np.where(df_90_3560['breed_n'].isin(breed_t1_list),1,0) 
print(df['breed_t1'].value_counts())

# pd.crosstab(df['breed_t1'].fillna('.'),df['breed_n'].fillna('.'))
# df['breed_n'].value_counts(10)

1    722
0    360
Name: breed_t6, dtype: int64
1    648
0    434
Name: breed_t5, dtype: int64
1    569
0    513
Name: breed_t4, dtype: int64
0    616
1    466
Name: breed_t3, dtype: int64
0    733
1    349
Name: breed_t2, dtype: int64
0    873
1    209
Name: breed_t1, dtype: int64


C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\Lo

# Get top 6 shunts for Pairwise

In [6]:
shunt_t6_list = df['shunt_ty'].value_counts().keys()[0:6].sort_values(ascending=False).tolist()
print(shunt_t6_list)
print(df['shunt_ty'].value_counts())

[6, 5, 4, 3, 2, 1]
1     295
2     208
3     165
4     131
5     118
6      23
7      22
8      18
9      11
11     10
13      8
12      8
10      8
14      5
15      4
18      4
16      4
17      4
19      3
22      3
21      3
25      2
23      2
24      2
20      2
44      1
27      1
40      1
41      1
31      1
36      1
39      1
42      1
33      1
26      1
32      1
34      1
29      1
45      1
37      1
30      1
28      1
35      1
43      1
Name: shunt_ty, dtype: int64


# Top hospital for pairwise

In [7]:
hosp_t6_list = df['hospital'].value_counts().keys()[0:6].sort_values(ascending=False).tolist()
print(hosp_t6_list)
print(df['hospital'].value_counts())

[11, 7, 6, 4, 1, 0]
4     178
7     156
6     117
0      94
11     89
1      64
3      62
9      62
5      58
8      58
12     50
10     48
2      46
Name: hospital, dtype: int64


# Create Top 10 Shunt Type variables based of of the one-way Chi-Squre Output

In [8]:

my_dfs = ['df_90_912p5', 'df_912p5_1825', 'df_1825_3560', 'df_90_3560']

shunt_t10_list = df['shunt_ty'].value_counts().keys()[0:10].sort_values(ascending=False).tolist()
df['shunt_t10'] = np.where(df['shunt_ty'].isin(shunt_t10_list),1,0) 
df_90_912p5['shunt_t10'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t10_list),1,0) 
df_912p5_1825['shunt_t10'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t10_list),1,0) 
df_1825_3560['shunt_t10'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t10_list),1,0) 
df_90_3560['shunt_t10'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t10_list),1,0) 
# print(df[['shunt_t10', 'shunt_ty']])
print(df['shunt_t10'].value_counts())

shunt_t9_list = df['shunt_ty'].value_counts().keys()[0:9].sort_values(ascending=False).tolist()
df['shunt_t9'] = np.where(df['shunt_ty'].isin(shunt_t9_list),1,0)
df_90_912p5['shunt_t9'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t9_list),1,0) 
df_912p5_1825['shunt_t9'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t9_list),1,0) 
df_1825_3560['shunt_t9'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t9_list),1,0) 
df_90_3560['shunt_t9'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t9_list),1,0) 
print(df['shunt_t9'].value_counts())

shunt_t8_list = df['shunt_ty'].value_counts().keys()[0:8].sort_values(ascending=False).tolist()
df['shunt_t8'] = np.where(df['shunt_ty'].isin(shunt_t8_list),1,0)
df_90_912p5['shunt_t8'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t8_list),1,0) 
df_912p5_1825['shunt_t8'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t8_list),1,0)
df_1825_3560['shunt_t8'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t8_list),1,0) 
df_90_3560['shunt_t8'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t8_list),1,0) 
print(df['shunt_t8'].value_counts())

shunt_t7_list = df['shunt_ty'].value_counts().keys()[0:7].sort_values(ascending=False).tolist()
df['shunt_t7'] = np.where(df['shunt_ty'].isin(shunt_t7_list),1,0) 
df_90_912p5['shunt_t7'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t7_list),1,0) 
df_912p5_1825['shunt_t7'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t7_list),1,0) 
df_1825_3560['shunt_t7'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t7_list),1,0)
df_90_3560['shunt_t7'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t7_list),1,0) 
print(df['shunt_t7'].value_counts())

shunt_t6_list = df['shunt_ty'].value_counts().keys()[0:6].sort_values(ascending=False).tolist()
df['shunt_t6'] = np.where(df['shunt_ty'].isin(shunt_t6_list),1,0)
df_90_912p5['shunt_t6'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t6_list),1,0) 
df_912p5_1825['shunt_t6'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t6_list),1,0) 
df_1825_3560['shunt_t6'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t6_list),1,0) 
df_90_3560['shunt_t6'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t6_list),1,0) 
print(df['shunt_t6'].value_counts())

shunt_t5_list = df['shunt_ty'].value_counts().keys()[0:5].sort_values(ascending=False).tolist()
df['shunt_t5'] = np.where(df['shunt_ty'].isin(shunt_t5_list),1,0)
df_90_912p5['shunt_t5'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t5_list),1,0) 
df_912p5_1825['shunt_t5'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t5_list),1,0) 
df_1825_3560['shunt_t5'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t5_list),1,0)
df_90_3560['shunt_t5'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t5_list),1,0) 
print(df['shunt_t5'].value_counts())

shunt_t4_list = df['shunt_ty'].value_counts().keys()[0:4].sort_values(ascending=False).tolist()
df['shunt_t4'] = np.where(df['shunt_ty'].isin(shunt_t4_list),1,0)
df_90_912p5['shunt_t4'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t4_list),1,0) 
df_912p5_1825['shunt_t4'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t4_list),1,0) 
df_1825_3560['shunt_t4'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t4_list),1,0)
df_90_3560['shunt_t4'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t4_list),1,0) 
print(df['shunt_t4'].value_counts())

shunt_t3_list = df['shunt_ty'].value_counts().keys()[0:3].sort_values(ascending=False).tolist()
df['shunt_t3'] = np.where(df['shunt_ty'].isin(shunt_t3_list),1,0)
df_90_912p5['shunt_t3'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t3_list),1,0) 
df_912p5_1825['shunt_t3'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t3_list),1,0) 
df_1825_3560['shunt_t3'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t3_list),1,0)
df_90_3560['shunt_t3'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t3_list),1,0) 
print(df['shunt_t3'].value_counts())

shunt_t2_list = df['shunt_ty'].value_counts().keys()[0:2].sort_values(ascending=False).tolist()
df['shunt_t2'] = np.where(df['shunt_ty'].isin(shunt_t2_list),1,0)
df_90_912p5['shunt_t2'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t2_list),1,0) 
df_912p5_1825['shunt_t2'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t2_list),1,0) 
df_1825_3560['shunt_t2'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t2_list),1,0)
df_90_3560['shunt_t2'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t2_list),1,0)
print(df['shunt_t2'].value_counts())

shunt_t1_list = df['shunt_ty'].value_counts().keys()[0:1].sort_values(ascending=False).tolist()
df['shunt_t1'] = np.where(df['shunt_ty'].isin(shunt_t1_list),1,0)
df_90_912p5['shunt_t1'] = np.where(df_90_912p5['shunt_ty'].isin(shunt_t1_list),1,0) 
df_912p5_1825['shunt_t1'] = np.where(df_912p5_1825['shunt_ty'].isin(shunt_t1_list),1,0) 
df_1825_3560['shunt_t1'] = np.where(df_1825_3560['shunt_ty'].isin(shunt_t1_list),1,0)
df_90_3560['shunt_t1'] = np.where(df_90_3560['shunt_ty'].isin(shunt_t1_list),1,0)
print(df['shunt_t1'].value_counts())



1    1001
0      81
Name: shunt_t10, dtype: int64
1    991
0     91
Name: shunt_t9, dtype: int64
1    980
0    102
Name: shunt_t8, dtype: int64
1    962
0    120
Name: shunt_t7, dtype: int64
1    940
0    142
Name: shunt_t6, dtype: int64
1    917
0    165
Name: shunt_t5, dtype: int64
1    799
0    283
Name: shunt_t4, dtype: int64
1    668
0    414
Name: shunt_t3, dtype: int64
0    579
1    503
Name: shunt_t2, dtype: int64
0    787
1    295
Name: shunt_t1, dtype: int64


C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCach

# One Way Frequency

In [9]:

my_list = cat.copy()
my_list.remove("breed_n")
my_list += ['breed_n']
# Remove 'age_lt_90', with data update
# my_list += ['age_lt_730','age_gt_1095','age_90_730','age_730_1095', 'age_2p5_5_10yr']
my_list += ['age_2p5_5_10yr']
# Adding in top breed and shunt types
my_list += ['shunt_t10', 'shunt_t9', 'shunt_t7', 'shunt_t6', 'shunt_t5', 'shunt_t4', 'shunt_t3', 'shunt_t2', 'shunt_t1', 'breed_t6', 'breed_t5', 'breed_t4', 'breed_t3', 'breed_t2', 'breed_t1']

my_cat_df_list = ['cat_df', 'df_90_912p5', 'df_912p5_1825', 'df_1825_3560', 'df_90_3560']
cat_df = pd.DataFrame()
# cat_df_90_730 = pd.DataFrame()
# cat_df_730_1095 = pd.DataFrame()
# cat_df_1095 = pd.DataFrame()
cat_df_90_912p5 = pd.DataFrame()
cat_df_912p5_1825 = pd.DataFrame()
cat_df_1825_3560 = pd.DataFrame()
cat_df_90_3560 = pd.DataFrame()




for i in my_list:
    cat_df = cat_df.append(chi_oneway(df,i,"df"))

for i in my_list:
    cat_df_90_912p5 = cat_df_90_912p5.append(chi_oneway(df_90_912p5,i,"df_90_912p5"))

for i in my_list:
    cat_df_912p5_1825 = cat_df_912p5_1825.append(chi_oneway(df_912p5_1825,i,"df_912p5_1825"))

for i in my_list:
    cat_df_1825_3560 = cat_df_1825_3560.append(chi_oneway(df_1825_3560,i,"df_1825_3560"))

for i in my_list:
    cat_df_90_3560 = cat_df_90_3560.append(chi_oneway(df_90_3560,i,"df_90_3560"))

print(cat_df.shape)
print(cat_df.head(2))




(199, 8)
  df_used         table  var_level  frequency   percent  cumfrequency  \
0      df  Table male_n        0.0        575  0.531423           575   
1      df  Table male_n        1.0        507  0.468577          1082   

   cumpercent  p_value  
0    0.531423  0.03871  
1    1.000000  0.03871  


# Two Way Frequency

In [10]:
my_list = cat.copy()
# remove 'age_lt_90', with data update
# my_list += ['age_lt_730','age_gt_1095','age_90_730','age_730_1095', 'age_2p5_5_10yr']
my_list += ['age_2p5_5_10yr']
# Adding in top breed and shunt types
my_list += ['shunt_t10', 'shunt_t9', 'shunt_t7', 'shunt_t6', 'shunt_t5', 'shunt_t4', 'shunt_t3', 'shunt_t2', 'shunt_t1', 'breed_t6', 'breed_t5', 'breed_t4',
 'breed_t3', 'breed_t2', 'breed_t1']

iter_list = list(itertools.combinations(my_list, 2))
iter_list

# my_ct_cat_df_names = ['cross_cat', 'cross_cat_90_730', 'cross_cat_730_1095', 'cross_cat_1095']
# my_ct_cat_df_list = [pd.DataFrame() for i in my_ct_cat_df_names]
# my_ct_cat_df = dict(zip(my_ct_cat_df_names, my_ct_cat_df_list))

# cross_cat_90_730 = pd.DataFrame()
# cross_cat_730_1095 = pd.DataFrame()
# cross_cat_1095 = pd.DataFrame()

# df_90_912p5 df_912p5_1825 df_1825_3560  df_90_3560

print(len(iter_list))
cross_cat = pd.DataFrame()
cross_cat_df_90_912p5 = pd.DataFrame()
cross_cat_df_912p5_1825 = pd.DataFrame()
cross_cat_df_1825_3560 = pd.DataFrame()
cross_cat_df_90_3560 = pd.DataFrame()

ct = 1

if ct ==1:
    f = open('df_ct.html','w')

my_len = (len(iter_list))
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, propz_df = chi_contingency(df,v1,v2,"1","chi_cont","df")
    cross_cat = cross_cat.append(cross_tmp)
    if ct ==1:
        my_html = propz_df.to_html()
        f.write("The Cross Tabular variables are {} and {}.".format(v1, v2))
        f.write(my_html)

if ct ==1:
    f.close()


# cross_cat_df_90_912p5
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, propz_df = chi_contingency(df_90_912p5,v1,v2,"1","chi_cont","df_90_912p5")
    cross_cat_df_90_912p5 = cross_cat_df_90_912p5.append(cross_tmp)


# df_912p5_1825
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, propz_df = chi_contingency(df_912p5_1825,v1,v2,"1","chi_cont","df_912p5_1825")
    cross_cat_df_912p5_1825 = cross_cat_df_912p5_1825.append(cross_tmp)

# cross_cat_df_1825_3560
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df_1825_3560,v1,v2,"1","chi_cont","df_1825_3560")
    cross_cat_df_1825_3560 = cross_cat_df_1825_3560.append(cross_tmp)


# df_90_3560
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df_90_3560,v1,v2,"1","chi_cont","df_90_3560")
    cross_cat_df_90_3560 = cross_cat_df_90_3560.append(cross_tmp)

print("cross_cat")
print(cross_cat.tail(5))
print("cross_cat_df_90_912p5")
print(cross_cat_df_90_912p5.tail(5))
print("cross_cat_df_912p5_1825")
print(cross_cat_df_912p5_1825.tail(5))
print("cross_cat_df_1825_3560")
print(cross_cat_df_1825_3560.tail(5))
print("cross_cat_df_90_3560")
print(cross_cat_df_90_3560.tail(5))

300
cross_cat
  df_used r_q_n  analysis research_q                      Table  Frequency  \
0      df     1  chi_cont          1  Table breed_t2 * breed_t1        733   
1      df     1  chi_cont          1  Table breed_t2 * breed_t1          0   
2      df     1  chi_cont          1  Table breed_t2 * breed_t1        140   
3      df     1  chi_cont          1  Table breed_t2 * breed_t1        209   
4      df     1  chi_cont          1  Table breed_t2 * breed_t1       1082   

      Percent Missing iv1 iv2  
0   67.744917       .   0   0  
1    0.000000       .   0   1  
2   12.939002       .   1   0  
3   19.316081       .   1   1  
4  100.000000       0   .   .  
cross_cat_df_90_912p5
       df_used r_q_n  analysis research_q                      Table  \
0  df_90_912p5     1  chi_cont          1  Table breed_t2 * breed_t1   
1  df_90_912p5     1  chi_cont          1  Table breed_t2 * breed_t1   
2  df_90_912p5     1  chi_cont          1  Table breed_t2 * breed_t1   
3  df_90_912p5 

# Descriptive Means first Overall then by Class

In [11]:

m_desc_df = pd.DataFrame()
# m_desc_df_90_730 = pd.DataFrame()
# m_desc_df_730_1095 = pd.DataFrame()
# m_desc_df_1095 = pd.DataFrame()

m_class_df = pd.DataFrame()
# m_class_df_90_730 = pd.DataFrame()
# m_class_df_730_1095 = pd.DataFrame()
# m_class_df_1095 = pd.DataFrame()

m_desc_df_90_912p5 = pd.DataFrame()
m_desc_df_912p5_1825 = pd.DataFrame()
m_desc_df_1825_3560 = pd.DataFrame()
m_desc_df_90_3560 = pd.DataFrame()

m_class_df = pd.DataFrame()
m_class_df_90_912p5 = pd.DataFrame()
m_class_df_912p5_1825 = pd.DataFrame()
m_class_df_1825_3560 = pd.DataFrame()
m_class_df_90_3560 = pd.DataFrame()


# df_90_912p5 df_912p5_1825 df_1825_3560  df_90_3560
for i in cont:
        desc_tmp = means_desc(df, i,"df")
        m_desc_df = m_desc_df.append(desc_tmp)
# df_90_912p5
        desc_tmp = means_desc(df_90_912p5, i,"df_90_912p5")
        m_desc_df_90_912p5 = m_desc_df_90_912p5.append(desc_tmp)
# df_912p5_1825
        desc_tmp = means_desc(df_912p5_1825, i,"df_912p5_1825")
        m_desc_df_912p5_1825 = m_desc_df_912p5_1825.append(desc_tmp)
# df_1825_3560
        desc_tmp = means_desc(df_1825_3560, i,"df_1825_3560")
        m_desc_df_1825_3560 = m_desc_df_1825_3560.append(desc_tmp)

# df_90_3560
        desc_tmp = means_desc(df_90_3560, i,"df_90_3560")
        m_desc_df_90_3560 = m_desc_df_90_3560.append(desc_tmp)


print("m_desc_df")
print(m_desc_df)
print("m_desc_df_90_912p5")
print(m_desc_df_90_912p5)
print("m_desc_df_912p5_1825")
print(m_desc_df_912p5_1825)
print("m_desc_df_1825_3560")
print(m_desc_df_1825_3560)
print("m_desc_df_90_3560")
print(m_desc_df_90_3560)

for i in cat:
    for j in cont:
        desc_class_tmp = means_desc_class(df,j, i,"df")
        m_class_df = m_class_df.append(desc_class_tmp)
# df_90_730
        desc_class_tmp = means_desc_class(df_90_912p5,j, i,"df_90_912p5")
        m_class_df_90_912p5 = m_class_df_90_912p5.append(desc_class_tmp)
# df_90_730
        desc_class_tmp = means_desc_class(df_912p5_1825,j, i,"df_912p5_1825")
        m_desc_df_912p5_1825 = m_desc_df_912p5_1825.append(desc_class_tmp)
# df_730_1095
        desc_class_tmp = means_desc_class(df_1825_3560,j, i,"df_1825_3560")
        m_class_df_1825_3560 = m_class_df_1825_3560.append(desc_class_tmp)
# df_1095
        desc_class_tmp = means_desc_class(df_90_3560,j, i,"df_90_3560")
        m_class_df_90_3560 = m_class_df_90_3560.append(desc_class_tmp)

print("m_class_df")       
print(m_class_df)
print("m_class_df_90_912p5")
print(m_class_df_90_912p5)
print("m_desc_df_912p5_1825")
print(m_desc_df_912p5_1825)
print("m_class_df_1825_3560")
print(m_class_df_1825_3560)
print("m_class_df_90_3560")
print(m_class_df_90_3560)


m_desc_df
  df_used  cont_var  nmiss       n         mean  median         std    min  \
0      df     age_d     69  1013.0  1012.245805  697.00  873.422122  93.00   
0      df     wt_kg     15  1067.0     4.828744    4.44    2.627325   0.83   
0      df  shunt_ty      0  1082.0     4.270795    3.00    5.631750   1.00   

      max     p25      p75  
0  3640.0  303.00  1512.00  
0    12.0    2.61     6.35  
0    45.0    1.00     5.00  
m_desc_df_90_912p5
       df_used  cont_var  nmiss      n        mean  median         std    min  \
0  df_90_912p5     age_d      0  592.0  396.983108   346.0  218.509231  93.00   
0  df_90_912p5     wt_kg      8  584.0    4.289623     3.8    2.529449   0.83   
0  df_90_912p5  shunt_ty      0  592.0    4.562500     3.0    5.643331   1.00   

     max     p25     p75  
0  910.0  215.75  559.25  
0   12.0    2.20    5.70  
0   44.0    2.00    5.00  
m_desc_df_912p5_1825
         df_used  cont_var  nmiss      n         mean    median         std  \
0  df_912

# Diffs with Tukeys Adjustment

In [12]:
# Set up affectionate naming for stratified LOB

# df_90_912p5 df_912p5_1825 df_1825_3560  df_90_3560

# lob_dict_in = {1:['df'], 2:['df_90_730'], 3:['df_730_1095'], 4:['df_1095']}
lob_dict_in = {1:['df'], 2:['df_90_912p5'], 3:['df_912p5_1825'], 4:['df_1825_3560'], 5:['df_90_3560']}
# lob_dict_in = {1:['df']}
lob_df = pd.DataFrame(lob_dict_in)
lob_df = lob_df.T.reset_index()
lob_df.rename(columns={0: 'ds_name', 'index': 'df_num'}, inplace = True)


# Set up affectionate Dict naming for DV
dv_dict_in = {1:['wt_kg'], 2:['age_d'], 3:['shunt_ty']}
dv_df = pd.DataFrame(dv_dict_in)
dv_df = dv_df.T.reset_index()
dv_df.rename(columns={0: 'dv_name', 'index':'dv_num'}, inplace=True)


# Set up Dict for Model ID to b merge from above dict
# lob_ds = {1:df}
# lob_ds = {1:df, 2:df_90_730, 3:df_730_1095, 4:df_1095}
lob_ds = {1:df, 2:df_90_912p5, 3:df_912p5_1825, 4:df_1825_3560, 5:df_90_3560}

# lob_ds = {1:df}

# If using age categoricals, can not use categorical datasets for ANOVA
# Er at age_lt_90
# ValueEr: must have at least one row in constraint matrix

res = stat()
data_dict = {}
mt_data = pd.DataFrame()

# Set normality statistics
normal_on = 0
if normal_on != 0:
    f = open('my_normality.html','w')


# Set up dict for Full DVs Full Rank but Catgetoricals with No PPSN
dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}
# Removing age as DV since actinga as IV
# dvs = {1:'wt_kg', 2:'pps_n'}
# Created new list with pps removed as IV for use as a DV
# for i in cat:
# for i in cat_nopps:
cat_nopps_s_b = cat_nopps.copy()
cat_nopps_s_b += ['shunt_t10', 'shunt_t9', 'shunt_t7', 'shunt_t6', 'shunt_t5', 'shunt_t4', 'shunt_t3', 'shunt_t2', 'shunt_t1', 'breed_t6', 'breed_t5', 'breed_t4', 'breed_t3', 'breed_t2', 'breed_t1']
# print("starting cat_nopps_s_b section")
for i in cat_nopps_s_b:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            anova_df = df.loc[:,[v1,i]].dropna()
            # res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)

                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                my_fig0 = plt
                encoded0 = fig_to_base64(my_fig0)
                my_html0 = '<img src="data:image/png;base64, {}">'.format(encoded0.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html0)

                # residuals histogram
                my_fig1 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig1.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                # encoded = base64.b64encode(tmpfile.getvalue()).decode('utf8') <img src='data:image/png;base64,{{}}'>
                my_html1 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html1)



            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])



# Get PPN as Sole IV with Wt and Age as DV
# Had to remove pps_n as a DV since it was an IV with dropna analysis
dvs = {1:'wt_kg', 2:'age_d'}
# Removing age as DV since age categoricals are acting as an IV
# print("starting cat_pps_n section")
# dvs = {1:'wt_kg'}
# for i in cat:
cat_pps_n = ['pps_n']
for i in cat_pps_n:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            anova_df = df.loc[:,[v1,i]].dropna()
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')

            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)
                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                plt.show()

                my_fig3 = plt
                encoded3 = fig_to_base64(my_fig3)
                my_html3 = '<img src="data:image/png;base64, {}">'.format(encoded3.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html3)

                # residuals histogram
                my_fig4 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig4.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                my_html4 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html4)


            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])




# Get Age LT 730 as Sole IV with Wt and wt and PPS
# Had to remove pps_n as a DV since it was an IV with dropna analysis
# dvs = {1:'wt_kg', 2:'pps_n'}
# # for i in cat:
# cat_age_lt_730_n = ['age_lt_730']
# for i in cat_age_lt_730_n:
#     for k1,v1 in dvs.items():
#         for k, v in lob_ds.items():
#             anova_df = df.loc[:,[v1,i]].dropna()
#             res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')

#             #Normality Section
#             if normal_on !=0:
#                 my_shapiro = shapiro(res.anova_std_residuals)
#                 my_shapiro = str(my_shapiro)
#                 my_ks = kstest(res.anova_std_residuals, 'norm')
#                 my_ks = str(my_ks)
#                 # residuals qq
#                 sm.qqplot(res.anova_std_residuals,line='45')
#                 plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
#                 plt.ylabel("Standardized Residuals")
#                 plt.show()

#                 my_fig5 = plt
#                 encoded5 = fig_to_base64(my_fig5)
#                 my_html5 = '<img src="data:image/png;base64, {}">'.format(encoded5.decode('utf-8'))
#                 f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
#                 f.write(my_html5)

#                 # residuals histogram
#                 my_fig6 = plt.figure()
#                 plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
#                 plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
#                 plt.ylabel('Frequency')
#                 plt.show()

#                 # Histogram Writing
#                 tmpfile = BytesIO()
#                 my_fig6.savefig(tmpfile, format='png')            
#                 encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
#                 my_html6 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
#                 f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
#                 f.write(my_html6)


#             tmp = res.tukey_summary
#             tmp['df_num'] = k
#             tmp['dv_num'] = k1
#             tmp['iv'] = i
#             data_dict[k] = tmp
#             mt_data = mt_data.append(data_dict[k])



diffs = pd.merge(mt_data, lob_df, on=["df_num"])
diffs = pd.merge(diffs, dv_df, on=["dv_num"])
diffs

if normal_on != 0:
    f.close()


# ANOVA of continuous vars with top 6 Breed IV

In [13]:

res = stat()
data_dict = {}
t5_breed_data = pd.DataFrame()

dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}

cat_breed = ['breed_n']
ds_breed = {1:breed_t6}

for i in cat_breed:
    for k1,v1 in dvs.items():
        for k, v in ds_breed.items():
            res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            t5_breed_data = t5_breed_data.append(data_dict[k])


breed_diffs = pd.merge(t5_breed_data, lob_df, on=["df_num"])
breed_diffs = pd.merge(breed_diffs, dv_df, on=["dv_num"])
breed_diffs

,group1,group2,Diff,Lower,Upper,q-value,p-value,df_num,dv_num,iv,ds_name,dv_name
0,53.0,35.0,2.210692,1.031520,3.389865,7.616228,0.001000,1,1,breed_n,df,wt_kg
1,53.0,62.0,0.057338,-1.517585,1.632262,0.147902,0.900000,1,1,breed_n,df,wt_kg
2,53.0,38.0,0.187714,-1.133443,1.508871,0.577207,0.900000,1,1,breed_n,df,wt_kg
3,53.0,47.0,1.942603,0.526961,3.358245,5.574676,0.001468,1,1,breed_n,df,wt_kg
4,53.0,61.0,2.461691,1.327890,3.595492,8.820347,0.001000,1,1,breed_n,df,wt_kg
5,35.0,62.0,2.153354,0.695487,3.611221,6.000486,0.001000,1,1,breed_n,df,wt_kg
6,35.0,38.0,2.398407,1.219234,3.577579,8.262937,0.001000,1,1,breed_n,df,wt_kg
7,35.0,47.0,4.153296,2.869148,5.437443,13.139141,0.001000,1,1,breed_n,df,wt_kg
8,35.0,61.0,0.250999,-0.713624,1.215621,1.057068,0.900000,1,1,breed_n,df,wt_kg
9,62.0,38.0,0.245053,-1.329871,1.819976,0.632104,0.900000,1,1,breed_n,df,wt_kg


# Z-Test for Proportionality

In [14]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# can we assume anything from our sample
significance = 0.025

def my_z_test(denom_in, num_in, denom_grp_1, denom_grp_2, grp_1, grp_2):
    proportion_data = pd.DataFrame()
    # Step 1 country is in index
    tmp = pd.crosstab(df[denom_in].fillna('.'),df[num_in].fillna('.'))
    thisFilter = tmp.filter('.')
    tmp.drop(thisFilter, inplace=True, axis=1)
    # tmp = tmp.drop(columns=['.'])
    tmp.reset_index(inplace=True)
    # Work on values

    tmp_1a = tmp.loc[tmp[denom_in] == denom_grp_1] # Country is 0
    tmp_1a = tmp_1a.drop([denom_in], axis=1)
    denom_1 = tmp_1a.loc[denom_grp_1].sum() # Country is 0
    # Get numerator from single row
    tmp_1b = tmp_1a[[grp_1]].reset_index() # pps_n is 1
    tmp_1b = tmp_1b.drop(['index'], axis=1)
    # Get only column and sum, if no sum will take full dataframe and not just numerator
    num_1 = tmp_1b.loc[0].sum()
    tup1 = (num_1, denom_1)
    # print("tup1")
    # print(tup1)
    pct_1 = num_1 / denom_1


    # Get single row to be analyzed for upcoming comparision
    tmp_2a = tmp.loc[tmp[denom_in] == denom_grp_2]
    tmp_2a = tmp_2a.drop([denom_in], axis=1)
    denom_2 = tmp_2a.loc[denom_grp_2].sum() # Country 1
    # Get numerator from single row e.g. pps_n 1 is first column, 2 is second column
    tmp_2b = tmp_2a[[grp_2]].reset_index() # pps_n is 1
    tmp_2b = tmp_2b.drop(['index'], axis=1)
    num_2 = tmp_2b.loc[0].sum() 
    tup2 = (num_2, denom_2)
    # print("tup2")
    # print(tup2)
    pct_2 = num_2 / denom_2
    sample_success_a, sample_size_a = tup1
    sample_success_b, sample_size_b = tup2
    successes = np.array([sample_success_a, sample_success_b])
    samples = np.array([sample_size_a, sample_size_b])

    # note, no need for a Ho value here - it's derived from the other parameters
    stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')


    # narrative = [('Comparing: Pct 1: %0.3f, Pct 2: %0.3f Num 1: %0.0f, Denom 1: %0.0f, Num 2: %0.0f, Denom 2: %0.0f'% (pct_1, pct_2, num_1, denom_1, num_2, denom_2) + 
    # ", V1/V2: " + denom_in + '/' + num_in + ', Den Lvls: %0.0f/%0.0f' % (denom_grp_1, denom_grp_2) + ', Num Lvls: %0.0f/%0.0f' % (grp_1, grp_2) + ', P_Val: %0.3f' % (p_value))]
    narrative = [(
          "V1/V2: " + denom_in + '/' + num_in + ', Den Lvls: %0.0f/%0.0f' % (denom_grp_1, denom_grp_2) + ', Num Lvls: %0.0f/%0.0f' % (grp_1, grp_2) 
        
        + 
  ', Comparing: Pct 1: %0.3f, Pct 2: %0.3f Num 1: %0.0f, Denom 1: %0.0f, Num 2: %0.0f, Denom 2: %0.0f'% (pct_1, pct_2, num_1, denom_1, num_2, denom_2) 
    + 
    ', P_Val: %0.3f' % (p_value)
    )]
    print(narrative)
    tmp_df = pd.DataFrame({'narrative' : narrative, 'p_value' : p_value})

    proportion_data = pd.concat([propz_df,tmp_df])
    # proportion_data = proportion_data.append(tmp_df)

    # return narrative, p_value
    return proportion_data


propz_df = pd.DataFrame()

# The first two are comparing pairwise for the first variable and should be different e.g. country 0/1, 0,2, 0/3
# The last two should be the same since we are comparing the same level of the second variable within the pairwise first variable e.g. country 0 pps 1 vs. country 1 pps 1

# Country 0 vs 1, 2, 3 | PPS 1
propz_df = my_z_test('country', 'pps_n', 0, 1, 1, 1) 
propz_df = my_z_test('country', 'pps_n', 0, 2, 1, 1) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 1), col val 2 (pps 1)
propz_df = my_z_test('country', 'pps_n', 0, 3, 1, 1) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 1), col val 2 (pps 1)
# Country 0 vs 1, 2, 3 | PPS 2
propz_df = my_z_test('country', 'pps_n', 0, 1, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 2), col val 2 (pps 2)
propz_df = my_z_test('country', 'pps_n', 0, 2, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 2), col val 2 (pps 2)
propz_df = my_z_test('country', 'pps_n', 0, 3, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 2), col val 2 (pps 2)
# Country 0 vs 1, 2, 3 | PPS 3
propz_df = my_z_test('country', 'pps_n', 0, 1, 3, 3) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 2, 3, 3) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 3, 3, 3) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 3), col val 2 (pps 3)
# Country 0 vs 1, 2, 3 | PPS 4
propz_df = my_z_test('country', 'pps_n', 0, 1, 4, 4) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 2, 4, 4) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 3, 4, 4) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 3), col val 2 (pps 3)
# Country 0 vs 1, 2, 3 | PPS 5
propz_df = my_z_test('country', 'pps_n', 0, 1, 5, 5) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 2, 5, 5) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 3), col val 2 (pps 3)
propz_df = my_z_test('country', 'pps_n', 0, 3, 5, 5) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 3), col val 2 (pps 3)
# Country 1 vs 2, 3 | PPS 1 and 2 and 3
propz_df = my_z_test('country', 'pps_n', 1, 2, 1, 1)
propz_df = my_z_test('country', 'pps_n', 1, 3, 1, 1)
propz_df = my_z_test('country', 'pps_n', 1, 2, 2, 2)
propz_df = my_z_test('country', 'pps_n', 1, 3, 2, 2)
propz_df = my_z_test('country', 'pps_n', 1, 2, 3, 3)
propz_df = my_z_test('country', 'pps_n', 1, 3, 3, 3)
propz_df = my_z_test('country', 'pps_n', 1, 2, 4, 4)
propz_df = my_z_test('country', 'pps_n', 1, 3, 4, 4)
propz_df = my_z_test('country', 'pps_n', 1, 2, 5, 5)
propz_df = my_z_test('country', 'pps_n', 1, 3, 5, 5)
# Country 0 vs 1, 2, 3 | PPS 1 and 2 and 3
propz_df = my_z_test('country', 'pps_n', 2, 3, 1, 1)
propz_df = my_z_test('country', 'pps_n', 2, 3, 2, 2)
propz_df = my_z_test('country', 'pps_n', 2, 3, 3, 3)
propz_df = my_z_test('country', 'pps_n', 2, 3, 4, 4)
propz_df = my_z_test('country', 'pps_n', 2, 3, 5, 5)

# Shunt Type top 6
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 1, 1) 
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 1, 1) 
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 1, 1)
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 3, 3)
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 3, 3) 
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 3, 3)
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 0, 1, 6, 6)
propz_df = my_z_test('country', 'shunt_ty', 0, 2, 6, 6)
propz_df = my_z_test('country', 'shunt_ty', 0, 3, 6, 6)

propz_df = my_z_test('country', 'shunt_ty', 1, 2, 1, 1)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 1, 1)
propz_df = my_z_test('country', 'shunt_ty', 1, 2, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 1, 2, 3, 3)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 3, 3)
propz_df = my_z_test('country', 'shunt_ty', 1, 2, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 1, 2, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 1, 2, 6, 6)
propz_df = my_z_test('country', 'shunt_ty', 1, 3, 6, 6)

propz_df = my_z_test('country', 'shunt_ty', 2, 3, 1, 1)
propz_df = my_z_test('country', 'shunt_ty', 2, 3, 2, 2)
propz_df = my_z_test('country', 'shunt_ty', 2, 3, 3, 3)
propz_df = my_z_test('country', 'shunt_ty', 2, 3, 4, 4)
propz_df = my_z_test('country', 'shunt_ty', 2, 3, 5, 5)
propz_df = my_z_test('country', 'shunt_ty', 2, 3, 6, 6)


# breed_t5_list = [62, 61, 53, 47, 38, 35]

propz_df = my_z_test('country', 'breed_n', 0, 1, 62, 62) 
propz_df = my_z_test('country', 'breed_n', 0, 2, 62, 62) 
propz_df = my_z_test('country', 'breed_n', 0, 3, 62, 62)
propz_df = my_z_test('country', 'breed_n', 0, 1, 61, 61)
propz_df = my_z_test('country', 'breed_n', 0, 2, 61, 61)
propz_df = my_z_test('country', 'breed_n', 0, 3, 61, 61)
propz_df = my_z_test('country', 'breed_n', 0, 1, 53, 53)
propz_df = my_z_test('country', 'breed_n', 0, 2, 53, 53) 
propz_df = my_z_test('country', 'breed_n', 0, 3, 53, 53)
propz_df = my_z_test('country', 'breed_n', 0, 1, 47, 47)
propz_df = my_z_test('country', 'breed_n', 0, 2, 47, 47)
propz_df = my_z_test('country', 'breed_n', 0, 3, 47, 47)
propz_df = my_z_test('country', 'breed_n', 0, 1, 38, 38)
propz_df = my_z_test('country', 'breed_n', 0, 2, 38, 38)
propz_df = my_z_test('country', 'breed_n', 0, 3, 38, 38)
propz_df = my_z_test('country', 'breed_n', 0, 1, 35, 35)
propz_df = my_z_test('country', 'breed_n', 0, 2, 35, 35)
propz_df = my_z_test('country', 'breed_n', 0, 3, 35, 35)

propz_df = my_z_test('country', 'breed_n', 1, 2, 62, 62)
propz_df = my_z_test('country', 'breed_n', 1, 3, 62, 62)
propz_df = my_z_test('country', 'breed_n', 1, 2, 61, 61)
propz_df = my_z_test('country', 'breed_n', 1, 3, 61, 61)
propz_df = my_z_test('country', 'breed_n', 1, 2, 53, 53)
propz_df = my_z_test('country', 'breed_n', 1, 3, 53, 53)
propz_df = my_z_test('country', 'breed_n', 1, 2, 47, 47)
propz_df = my_z_test('country', 'breed_n', 1, 3, 47, 47)
propz_df = my_z_test('country', 'breed_n', 1, 2, 38, 38)
propz_df = my_z_test('country', 'breed_n', 1, 3, 38, 38)
propz_df = my_z_test('country', 'breed_n', 1, 2, 35, 35)
propz_df = my_z_test('country', 'breed_n', 1, 3, 35, 35)

propz_df = my_z_test('country', 'breed_n', 2, 3, 62, 62)
propz_df = my_z_test('country', 'breed_n', 2, 3, 61, 61)
propz_df = my_z_test('country', 'breed_n', 2, 3, 53, 53)
propz_df = my_z_test('country', 'breed_n', 2, 3, 47, 47)
propz_df = my_z_test('country', 'breed_n', 2, 3, 38, 38)
propz_df = my_z_test('country', 'breed_n', 2, 3, 35, 35)


# 'age_2p5_5_10yr' 1 2 3 4
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 1, 1, 1) 
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 2, 1, 1) 
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 3, 1, 1)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 1, 2, 2)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 2, 2, 2)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 3, 2, 2)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 1, 3, 3)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 2, 3, 3) 
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 3, 3, 3)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 1, 4, 4)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 2, 4, 4)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 0, 3, 4, 4)

propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 2, 1, 1)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 3, 1, 1)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 2, 2, 2)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 3, 2, 2)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 2, 3, 3)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 3, 3, 3)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 2, 4, 4)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 1, 3, 4, 4)

propz_df = my_z_test('country', 'age_2p5_5_10yr', 2, 3, 1, 1)
propz_df = my_z_test('country', 'age_2p5_5_10yr', 2, 3, 2, 2)
print('my runtime')
propz_df = my_z_test('country', 'age_2p5_5_10yr', 2, 3, 4, 4)
print('keep going')
propz_df = my_z_test('country', 'age_2p5_5_10yr', 2, 3, 3, 3)



# Hospital
# [11, 7, 6, 4, 1, 0]



# hospital 11 vs 7 6 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 1, 1) 
propz_df = my_z_test('hospital', 'pps_n', 11, 6, 1, 1) 
propz_df = my_z_test('hospital', 'pps_n', 11, 4, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 11, 1, 1, 1) 
propz_df = my_z_test('hospital', 'pps_n', 11, 0, 1, 1)

propz_df = my_z_test('hospital', 'pps_n', 11, 7, 2, 2) 
propz_df = my_z_test('hospital', 'pps_n', 11, 6, 2, 2) 
propz_df = my_z_test('hospital', 'pps_n', 11, 4, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 11, 1, 2, 2) 
propz_df = my_z_test('hospital', 'pps_n', 11, 0, 2, 2)

propz_df = my_z_test('hospital', 'pps_n', 11, 7, 3, 3) 
propz_df = my_z_test('hospital', 'pps_n', 11, 6, 3, 3) 
propz_df = my_z_test('hospital', 'pps_n', 11, 4, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 11, 1, 3, 3) 
propz_df = my_z_test('hospital', 'pps_n', 11, 0, 3, 3)

propz_df = my_z_test('hospital', 'pps_n', 11, 7, 4, 4) 
propz_df = my_z_test('hospital', 'pps_n', 11, 6, 4, 4) 
propz_df = my_z_test('hospital', 'pps_n', 11, 4, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 11, 1, 4, 4) 
propz_df = my_z_test('hospital', 'pps_n', 11, 0, 4, 4)

propz_df = my_z_test('hospital', 'pps_n', 11, 7, 5, 5) 
propz_df = my_z_test('hospital', 'pps_n', 11, 6, 5, 5) 
propz_df = my_z_test('hospital', 'pps_n', 11, 4, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 11, 1, 5, 5) 
propz_df = my_z_test('hospital', 'pps_n', 11, 0, 5, 5)

# hospital 7 vs 6 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'pps_n', 7, 6, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 7, 4, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 7, 1, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 7, 0, 1, 1)

propz_df = my_z_test('hospital', 'pps_n', 7, 6, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 7, 4, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 7, 1, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 7, 0, 2, 2)

propz_df = my_z_test('hospital', 'pps_n', 7, 6, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 7, 4, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 7, 1, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 7, 0, 3, 3)

propz_df = my_z_test('hospital', 'pps_n', 7, 6, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 7, 4, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 7, 1, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 7, 0, 4, 4)

propz_df = my_z_test('hospital', 'pps_n', 7, 6, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 7, 4, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 7, 1, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 7, 0, 5, 5)


# hospital 6 vs 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'pps_n', 6, 4, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 6, 1, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 6, 0, 1, 1)

propz_df = my_z_test('hospital', 'pps_n', 6, 4, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 6, 1, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 6, 0, 2, 2)

propz_df = my_z_test('hospital', 'pps_n', 6, 4, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 6, 1, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 6, 0, 3, 3)

propz_df = my_z_test('hospital', 'pps_n', 6, 4, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 6, 1, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 6, 0, 4, 4)

propz_df = my_z_test('hospital', 'pps_n', 6, 4, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 6, 1, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 6, 0, 5, 5)

# hospital 4 vs 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'pps_n', 4, 1, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 4, 0, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 4, 1, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 4, 0, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 4, 1, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 4, 0, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 4, 1, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 4, 0, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 4, 1, 5, 5)
propz_df = my_z_test('hospital', 'pps_n', 4, 0, 5, 5)

# hospital 1 vs 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'pps_n', 1, 0, 1, 1)
propz_df = my_z_test('hospital', 'pps_n', 1, 0, 2, 2)
propz_df = my_z_test('hospital', 'pps_n', 1, 0, 3, 3)
propz_df = my_z_test('hospital', 'pps_n', 1, 0, 4, 4)
propz_df = my_z_test('hospital', 'pps_n', 1, 0, 5, 5)





# Shunt Type top 6


# hospital 11 vs 7 6 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 1, 1) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 1, 1) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 1, 1) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 1, 1)

propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 2, 2) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 2, 2) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 2, 2) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 2, 2)

propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 3, 3) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 3, 3) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 3, 3) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 3, 3)

propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 4, 4) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 4, 4) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 4, 4) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 4, 4)

propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 5, 5) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 5, 5) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 5, 5) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 5, 5)

propz_df = my_z_test('hospital', 'shunt_ty', 11, 7, 6, 6) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 6, 6, 6) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 4, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 11, 1, 6, 6) 
propz_df = my_z_test('hospital', 'shunt_ty', 11, 0, 6, 6)

# hospital 7 vs 6 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 1, 1)

propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 2, 2)

propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 3, 3)

propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 4, 4)

propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 5, 5)

propz_df = my_z_test('hospital', 'shunt_ty', 7, 6, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 4, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 1, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 7, 0, 6, 6)

# hospital 6 vs 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 1, 1)

propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 2, 2)

propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 3, 3)

propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 4, 4)

propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 5, 5)

propz_df = my_z_test('hospital', 'shunt_ty', 6, 4, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 1, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 6, 0, 6, 6)

# hospital 4 vs 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 1, 6, 6)
propz_df = my_z_test('hospital', 'shunt_ty', 4, 0, 6, 6)

# hospital 1 vs 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 1, 1)
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 2, 2)
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 3, 3)
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 4, 4)
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 5, 5)
propz_df = my_z_test('hospital', 'shunt_ty', 1, 0, 6, 6)



# hospital 11 vs 7 6 4 1 0 | breed_t5_list = [62, 61, 53, 47, 38, 35]
propz_df = my_z_test('hospital', 'breed_n', 11, 7, 62, 62) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 62, 62) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 62, 62) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 62, 62)

propz_df = my_z_test('hospital', 'breed_n', 11, 7, 61, 61) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 61, 61) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 61, 61) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 61, 61)

propz_df = my_z_test('hospital', 'breed_n', 11, 7, 53, 53) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 53, 53) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 53, 53) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 53, 53)

propz_df = my_z_test('hospital', 'breed_n', 11, 7, 47, 47) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 47, 47) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 47, 47) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 47, 47)

propz_df = my_z_test('hospital', 'breed_n', 11, 7, 38, 38) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 38, 38) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 38, 38) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 38, 38)

propz_df = my_z_test('hospital', 'breed_n', 11, 7, 35, 35) 
propz_df = my_z_test('hospital', 'breed_n', 11, 6, 35, 35) 
propz_df = my_z_test('hospital', 'breed_n', 11, 4, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 11, 1, 35, 35) 
propz_df = my_z_test('hospital', 'breed_n', 11, 0, 35, 35)

# hospital 7 vs 6 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'breed_n', 7, 6, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 62, 62)

propz_df = my_z_test('hospital', 'breed_n', 7, 6, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 61, 61)

propz_df = my_z_test('hospital', 'breed_n', 7, 6, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 53, 53)

propz_df = my_z_test('hospital', 'breed_n', 7, 6, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 47, 47)

propz_df = my_z_test('hospital', 'breed_n', 7, 6, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 38, 38)

propz_df = my_z_test('hospital', 'breed_n', 7, 6, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 7, 4, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 7, 1, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 7, 0, 35, 35)

# hospital 6 vs 4 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'breed_n', 6, 4, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 62, 62)

propz_df = my_z_test('hospital', 'breed_n', 6, 4, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 61, 61)

propz_df = my_z_test('hospital', 'breed_n', 6, 4, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 53, 53)

propz_df = my_z_test('hospital', 'breed_n', 6, 4, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 47, 47)

propz_df = my_z_test('hospital', 'breed_n', 6, 4, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 38, 38)

propz_df = my_z_test('hospital', 'breed_n', 6, 4, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 6, 1, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 6, 0, 35, 35)

# hospital 4 vs 1 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 4, 1, 35, 35)
propz_df = my_z_test('hospital', 'breed_n', 4, 0, 35, 35)

# hospital 1 vs 0 | PPS 1 2 3 4 5
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 62, 62)
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 61, 61)
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 53, 53)
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 47, 47)
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 38, 38)
propz_df = my_z_test('hospital', 'breed_n', 1, 0, 35, 35)



# 'age_2p5_5_10yr' 1 2 3 4

# hospital 11 vs 7 6 4 1 0 | age_2p5_5_10yr
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 7, 1, 1) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 6, 1, 1) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 4, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 1, 1, 1) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 0, 1, 1)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 7, 2, 2) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 6, 2, 2) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 4, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 1, 2, 2) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 0, 2, 2)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 7, 3, 3) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 6, 3, 3) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 4, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 1, 3, 3) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 0, 3, 3)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 7, 4, 4) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 6, 4, 4) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 4, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 1, 4, 4) 
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 11, 0, 4, 4)


# hospital 7 vs 6 4 1 0 | age_2p5_5_10yr
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 6, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 4, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 1, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 0, 1, 1)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 6, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 4, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 1, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 0, 2, 2)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 6, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 4, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 1, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 0, 3, 3)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 6, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 4, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 1, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 7, 0, 4, 4)



# hospital 6 vs 4 1 0 | age_2p5_5_10yr
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 4, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 1, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 0, 1, 1)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 4, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 1, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 0, 2, 2)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 4, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 1, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 0, 3, 3)

propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 4, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 1, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 6, 0, 4, 4)

# hospital 4 vs 1 0 | age_2p5_5_10yr
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 1, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 0, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 1, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 0, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 1, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 0, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 1, 4, 4)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 4, 0, 4, 4)


# hospital 1 vs 0 | age_2p5_5_10yr
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 1, 0, 1, 1)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 1, 0, 2, 2)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 1, 0, 3, 3)
propz_df = my_z_test('hospital', 'age_2p5_5_10yr', 1, 0, 4, 4)


['V1/V2: country/pps_n, Den Lvls: 0/1, Num Lvls: 1/1, Comparing: Pct 1: 0.271, Pct 2: 0.258 Num 1: 220, Denom 1: 812, Num 2: 16, Denom 2: 62, P_Val: 0.826']
['V1/V2: country/pps_n, Den Lvls: 0/2, Num Lvls: 1/1, Comparing: Pct 1: 0.271, Pct 2: 0.292 Num 1: 220, Denom 1: 812, Num 2: 14, Denom 2: 48, P_Val: 0.754']
['V1/V2: country/pps_n, Den Lvls: 0/3, Num Lvls: 1/1, Comparing: Pct 1: 0.271, Pct 2: 0.122 Num 1: 220, Denom 1: 812, Num 2: 19, Denom 2: 156, P_Val: 0.000']
['V1/V2: country/pps_n, Den Lvls: 0/1, Num Lvls: 2/2, Comparing: Pct 1: 0.087, Pct 2: 0.097 Num 1: 71, Denom 1: 812, Num 2: 6, Denom 2: 62, P_Val: 0.803']
['V1/V2: country/pps_n, Den Lvls: 0/2, Num Lvls: 2/2, Comparing: Pct 1: 0.087, Pct 2: 0.146 Num 1: 71, Denom 1: 812, Num 2: 7, Denom 2: 48, P_Val: 0.171']
['V1/V2: country/pps_n, Den Lvls: 0/3, Num Lvls: 2/2, Comparing: Pct 1: 0.087, Pct 2: 0.051 Num 1: 71, Denom 1: 812, Num 2: 8, Denom 2: 156, P_Val: 0.131']
['V1/V2: country/pps_n, Den Lvls: 0/1, Num Lvls: 3/3, Comparin

C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


['V1/V2: country/age_2p5_5_10yr, Den Lvls: 1/2, Num Lvls: 4/4, Comparing: Pct 1: 0.000, Pct 2: 0.000 Num 1: 0, Denom 1: 62, Num 2: 0, Denom 2: 48, P_Val: nan']
['V1/V2: country/age_2p5_5_10yr, Den Lvls: 1/3, Num Lvls: 4/4, Comparing: Pct 1: 0.000, Pct 2: 0.000 Num 1: 0, Denom 1: 62, Num 2: 0, Denom 2: 156, P_Val: nan']
['V1/V2: country/age_2p5_5_10yr, Den Lvls: 2/3, Num Lvls: 1/1, Comparing: Pct 1: 0.604, Pct 2: 0.494 Num 1: 29, Denom 1: 48, Num 2: 77, Denom 2: 156, P_Val: 0.180']
['V1/V2: country/age_2p5_5_10yr, Den Lvls: 2/3, Num Lvls: 2/2, Comparing: Pct 1: 0.167, Pct 2: 0.179 Num 1: 8, Denom 1: 48, Num 2: 28, Denom 2: 156, P_Val: 0.839']
my runtime
['V1/V2: country/age_2p5_5_10yr, Den Lvls: 2/3, Num Lvls: 4/4, Comparing: Pct 1: 0.000, Pct 2: 0.000 Num 1: 0, Denom 1: 48, Num 2: 0, Denom 2: 156, P_Val: nan']
keep going
['V1/V2: country/age_2p5_5_10yr, Den Lvls: 2/3, Num Lvls: 3/3, Comparing: Pct 1: 0.167, Pct 2: 0.237 Num 1: 8, Denom 1: 48, Num 2: 37, Denom 2: 156, P_Val: 0.303']
['V

# Write to Excel

In [15]:
# my_sheets = ['cat_one' 'cross_cat' 'm_desc' 'm_class' 'diffs' 'breed_diffs']
# lob_ds = {1:df, 2:df_90_912p5, 3:df_912p5_1825, 4:df_1825_3560, 5:df_90_3560}
writer = pd.ExcelWriter('ephss_out_20230116.xlsx', engine = 'xlsxwriter')
# for i in my_sheets:
#     [i].to_excel(writer,sheet_name=i)

# Data In
df.to_excel(writer,sheet_name='df')
df_90_912p5.to_excel(writer,sheet_name='df_90_912p5')
df_912p5_1825.to_excel(writer,sheet_name='df_912p5_1825')
df_1825_3560.to_excel(writer,sheet_name='df_1825_3560')
df_90_3560.to_excel(writer,sheet_name='df_90_3560')
# One Way Freq
cat_df.to_excel(writer,sheet_name='cat_df')
cat_df_90_912p5.to_excel(writer,sheet_name='cat_df_90_912p5')
cat_df_912p5_1825.to_excel(writer,sheet_name='cat_df_912p5_1825')
cat_df_1825_3560.to_excel(writer,sheet_name='cat_df_1825_3560')
cat_df_90_3560.to_excel(writer,sheet_name='cat_df_90_3560')
# Cross Tab Frequency
cross_cat.to_excel(writer,sheet_name='cross_cat')
cross_cat_df_90_912p5.to_excel(writer,sheet_name='cross_cat_df_90_912p5')
cross_cat_df_912p5_1825.to_excel(writer,sheet_name='cross_cat_df_912p5_1825')
cross_cat_df_1825_3560.to_excel(writer,sheet_name='cross_cat_df_1825_3560')
cross_cat_df_90_3560.to_excel(writer,sheet_name='cross_cat_df_90_3560')
# Means overall
m_desc_df.to_excel(writer,sheet_name='m_desc_df')
m_desc_df_90_912p5.to_excel(writer,sheet_name='m_desc_df_90_912p5')
m_desc_df_912p5_1825.to_excel(writer,sheet_name='m_desc_df_912p5_1825')
m_desc_df_1825_3560.to_excel(writer,sheet_name='m_desc_df_1825_3560')
m_desc_df_90_3560.to_excel(writer,sheet_name='m_desc_df_90_3560')
# Means with class
m_class_df.to_excel(writer,sheet_name='m_class_df')
m_class_df_90_912p5.to_excel(writer,sheet_name='m_class_df_90_912p5')
m_class_df_912p5_1825.to_excel(writer,sheet_name='m_class_df_912p5_1825')
m_class_df_1825_3560.to_excel(writer,sheet_name='m_class_df_1825_3560')
m_class_df_90_3560.to_excel(writer,sheet_name='m_class_df_90_3560')
# Diffs
diffs.to_excel(writer,sheet_name='diffs')
breed_diffs.to_excel(writer,sheet_name='breed_diffs')
# Diffs
propz_df.to_excel(writer,sheet_name='propz_df')

writer.save()


# Single df write
# my_sheets = ['df']
# writer = pd.ExcelWriter('df.xlsx', engine = 'xlsxwriter')
# df.to_excel(writer,sheet_name='data')
# writer.save()


# Prob TTDIST from Excel Proportionality

In [16]:
# From Excel Example
# https://stackoverflow.com/questions/56991534/excels-tdist-in-scipy
# from scipy import stats
stats.t.sf(.66299299354, df=9999999999999) * 2


0.5073350414125459

In [ ]:
# https://github.com/sonalake/blog-hypothesis-testing/blob/master/examples/proportions/2-sample-z-test.py
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# can we assume anything from our sample
significance = 0.025

# our samples;  note - the samples do not need to be the same size

# my_a = (16, 62)
# my_b = (220, 812)
# sample_success_a, sample_size_a = my_a
# sample_success_b, sample_size_b = my_b

# here 410 samples out of 500 were "ok"
sample_success_a, sample_size_a = (16, 62)
# here 379 samples out of 400 were "ok"
sample_success_b, sample_size_b = (220, 812)

# check our sample against Ho for Ha != Ho
successes = np.array([sample_success_a, sample_success_b])
samples = np.array([sample_size_a, sample_size_b])

# note, no need for a Ho value here - it's derived from the other parameters
stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')

# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))

if p_value > significance:
    print("Fail to reject the null hypothesis - we have nothing else to say")
else:
    print("Reject the null hypothesis - suggest the alternative hypothesis is true")

z_stat: -0.220, p_value: 0.826
Fail to reject the null hypothesis - we have nothing else to say


In [ ]:
# Step 1 country is in index
tmp = pd.crosstab(df['country'].fillna('.'),df['pps_n'].fillna('.'))
tmp = tmp.drop(columns=['.'])
tmp.reset_index(inplace=True)
# Get country out of index
# print("tmp_a2")
# print(tmp_a.head(10))


# Get single row to be analyzed for upcoming comparision
tmp_1a = tmp.loc[tmp['country'] == 0]
tmp_1a = tmp_1a.drop(['country'], axis=1)
# print("tmp_b")
# print(tmp_b.head(10))
denom_1 = tmp_1a.loc[0].sum() # Country 0
# print("denom_0")
# print(denom_0)
# type(denom_0)

# Get numerator from single row
tmp_1b = tmp_1a[[1]].reset_index() # pps_n is 1
tmp_1b = tmp_1b.drop(['index'], axis=1)
# print("tmp_c")
# print(tmp_c.head(10))
# Get only column and sum, if no sum will take full dataframe and not just numerator
num_1 = tmp_1b.loc[0].sum() 
# print("num_0")
# print(num_0)
# print(type(num_0))
tup1 = (num_1, denom_1)
print("tup1")
print(tup1)
type(tup1)


# Get single row to be analyzed for upcoming comparision
tmp_2a = tmp.loc[tmp['country'] == 1]
tmp_2a = tmp_2a.drop(['country'], axis=1)
# print("tmp_2a")
# print(tmp_2a.head(10))
denom_2 = tmp_2a.loc[1].sum() # Country 1
print("denom_2")
print(denom_2)
# Get numerator from single row e.g. pps_n 1 is first column, 2 is second column
tmp_2b = tmp_2a[[1]].reset_index() # pps_n is 1
tmp_2b = tmp_2b.drop(['index'], axis=1)
num_2 = tmp_2b.loc[0].sum() 
tup2 = (num_2, denom_2)
print("tup2")
print(tup2)
type(tup2)



sample_success_a, sample_size_a = tup1
# here 379 samples out of 400 were "ok"
sample_success_b, sample_size_b = tup2

# check our sample against Ho for Ha != Ho
successes = np.array([sample_success_a, sample_success_b])
samples = np.array([sample_size_a, sample_size_b])

# note, no need for a Ho value here - it's derived from the other parameters
stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')

# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))

if p_value > significance:
    print("Fail to reject the null hypothesis - we have nothing else to say")
else:
    print("Reject the null hypothesis - suggest the alternative hypothesis is true")

tup1
(220, 812)
denom_2
62
tup2
(16, 62)
z_stat: 0.220, p_value: 0.826
Fail to reject the null hypothesis - we have nothing else to say
